# Required Libraries

In [1]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.5 MB/s et

In [2]:
import os
import json
import torch
import chromadb
import requests
import numpy as np
from PIL import Image
from io import BytesIO
from chromadb.config import Settings
from transformers import CLIPVisionModel, RobertaModel, AutoTokenizer, CLIPFeatureExtractor

# Download Data

In [3]:
!git clone https://github.com/FaSha20/Natural-Language-Processing-Projects

Cloning into 'Natural-Language-Processing-Projects'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 90 (delta 27), reused 46 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (90/90), 8.11 MiB | 9.76 MiB/s, done.
Resolving deltas: 100% (27/27), done.


In [4]:
with open("/content/Natural-Language-Processing-Projects/MultimodalRAG/scientists_data/scintists_data_with_text_chunks.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print(len(data), "records loaded")

300 records loaded


# Build Embeddings and DATABASE

## Load Pre-trained Encoder Model

In [5]:
vision_encoder = CLIPVisionModel.from_pretrained('SajjadAyoubi/clip-fa-vision')
preprocessor = CLIPFeatureExtractor.from_pretrained('SajjadAyoubi/clip-fa-vision')
text_encoder = RobertaModel.from_pretrained('SajjadAyoubi/clip-fa-text')
tokenizer = AutoTokenizer.from_pretrained('SajjadAyoubi/clip-fa-text')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/350M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/clip/feature_extraction_clip.py:30: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/354 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

## Encoding Functions

In [6]:
# ---- Text Encoder ----
def embed_text(text: str):
    text_embedding = text_encoder(**tokenizer(text, return_tensors='pt')).pooler_output
    return text_embedding[0]

# ---- Image Encoder ----
def embed_image(url: str):
    try:
        headers = {"User-Agent": "ImageDatasetScript/1.0 (email@gmail.com)"}
        response = requests.get(url, headers=headers, timeout=10)
        image = Image.open(BytesIO(response.content)).convert("RGB")
        image_embedding = vision_encoder(**preprocessor(image, return_tensors='pt')).pooler_output
        return image_embedding[0]
    except Exception as e:
        return None

## Build the DATABASE

In [7]:
PERSIST_DIR = "/content/chroma_scientists_db"
os.makedirs(PERSIST_DIR, exist_ok=True)

client = chromadb.PersistentClient(path=PERSIST_DIR, settings=Settings(allow_reset=True))

text_db  = client.get_or_create_collection(name="scientists_text",  metadata={"hnsw:space": "cosine"})
image_db = client.get_or_create_collection(name="scientists_image", metadata={"hnsw:space": "cosine"})

In [8]:
def upsert_pair(idx, name, text, image_url, t_emb, i_emb):
    meta = {
        "pair_id": idx,
        "name": name,
        "text": text,
        "image_url": image_url if image_url is not None else "",
    }

    if t_emb is not None:
        text_db.add(
            ids=[f"text-{idx}"],
            embeddings=[t_emb.tolist()],
            metadatas=[meta],
        )

    if i_emb is not None:
        image_db.add(
            ids=[f"img-{idx}"],
            embeddings=[i_emb.tolist()],
            metadatas=[meta],
        )

In [9]:
for idx, person in enumerate(data):
    name = person.get("name")

    text = person.get("text_chunk", "").replace("\u200c", "")
    t_emb = embed_text(text)

    image_urls = list(person.get("image", {}).values())
    img_url = None
    for url in image_urls:
        if url:
            i_emb = embed_image(url)
            if i_emb is not None:
              img_url = url
              break

    upsert_pair(idx, name, text, img_url, t_emb, i_emb)

## Retrieval Functions

In [10]:
def search_by_text(query: str, k=3):
    q_emb = embed_text(query)
    res = text_db.query(query_embeddings=[q_emb.tolist()], n_results=k, include=["metadatas", "distances"])
    return res

def search_by_image(image_url: str, k=5):
    q_emb = embed_image(image_url)
    if q_emb is None:
        return None
    res = image_db.query(query_embeddings=[q_emb.tolist()], n_results=k, include=["metadatas", "distances"])
    return res

## Test an example

In [11]:
res = search_by_text("چه کسی شرح کتاب مابعدالطبیعه ارسطو را نوشت؟", k=7)
for meta, dist in zip(res["metadatas"][0], res["distances"][0]):
    print("name:", meta["name"], "score:", 1-dist)
    print("text:", meta["text"])
    print("image:", meta["image_url"], "\n")

name: بنیامین بن موسی نهاوندی score: 0.8839473724365234
text: بنیامین بن موسی نهاوندی، حکیم، فیلسوف، عالم دینی، پزشک و عالم یهودی برجسته، در حدود قرن دوم شمسی در نهاوند واقع در استان همدان چشم به جهان گشود. او که در دوران قرون وسطی می زیست، نقشی اساسی در تکمیل و توسعه مبانی فرقه قرائیم ایفا نمود، جنبشی که توسط عنان ابن داوود پایه گذاری شده بود.  از جمله آثار ارزشمند او می توان به شروحی بر عهد عتیق و نوشته هایی به زبان عبری اشاره کرد.  نهاوندی با تاکید بر اهمیت تفکر آزاد و تحقیق، استدلال می کرد که تحقیق یک وظیفه است و اشتباهات ناشی از آن گناه محسوب نمی شود.
image:  

name: محمد بن طاهر بن بهرام سجستانی سیستانی score: 0.8695303201675415
text: محمد بن طاهر بن بهرام سجستانی سیستانی، معروف به "منطقی"، در حدود سال ۲۹۱ هجری شمسی در سیستان چشم به جهان گشود. این فیلسوف و اندیشمند برجسته سده چهارم هجری، تحصیلات خود را در زمینه فلسفه و منطق در بغداد و نزد اساتیدی چون ابوبشر متی بن یونس و ابوزکریا یحی بن عدی گذراند. سجستانی، علاوه بر فعالیتهای علمی، از حمایت سیاسی حاکمان محلی و دیلمی نیز برخوردار 

In [12]:
query_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/e/e6/Birjandimanuscript.jpg/220px-Birjandimanuscript.jpg"
res = search_by_image(query_url, k=5)
for meta, dist in zip(res["metadatas"][0], res["distances"][0]):
    print("name:", meta["name"], "score:", 1-dist)
    print("text:", meta["text"])
    print("image:", meta["image_url"], "\n")

name: نظام الدین عبدالعلی بیرجندی score: 1.0000001192092896
text: نظامالدین عبدالعلی بیرجندی، معروف به فاضل بیرجندی و محقق بیرجندی، دانشمند برجسته عصر صفوی بود. وی که در بیرجند واقع در خراسان جنوبی دیده به جهان گشود، در زمینههای ریاضیات و ستارهشناسی تبحر داشت. بیرجندی آثار متعددی از جمله "اسطرلاب"، "مختصر الهیئه" و "شرح التذکرة النصیریة فی الهیئة" را به رشته تحریر درآورد. او نزد اساتید بزرگی همچون غیاثالدین جمشید کاشانی و کمالالدین قنوی به تحصیل علوم مختلف پرداخت و سرانجام در سال ۹۳۴ هجری شمسی در بیرجند دار فانی را وداع گفت و در بجد به خاک سپرده شد.
image: https://upload.wikimedia.org/wikipedia/commons/thumb/e/e6/Birjandimanuscript.jpg/220px-Birjandimanuscript.jpg 

name: علی بن عباس مجوسی اهوازی score: 0.8311042189598083
text: علی بن عباس مجوسی اهوازی، پزشک و روانشناس برجسته سده چهارم هجری (دهم میلادی)، در شهر اهواز در ایران متولد شد. او که با القابی همچون "ابن المجوس" و "مجوسی اهوازی" نیز شناخته میشد، اثر ارزشمندی به نام "کتاب ملکی" از خود به یادگار گذاشته است. وی در حدود سال ۳۸۳ هجر

# Generator

In [13]:
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration

dtype = torch.float16
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf", trust_remote_code=True)
llava = LlavaForConditionalGeneration.from_pretrained("bczhou/tiny-llava-v1-hf", torch_dtype=dtype, device_map="auto", resume_download=True)
llava.eval()

processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/674 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

model-00002-of-00002.safetensors:   0%|          | 0.00/661M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

LlavaForConditionalGeneration(
  (model): LlavaModel(
    (vision_tower): CLIPVisionModel(
      (vision_model): CLIPVisionTransformer(
        (embeddings): CLIPVisionEmbeddings(
          (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
          (position_embedding): Embedding(577, 1024)
        )
        (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (encoder): CLIPEncoder(
          (layers): ModuleList(
            (0-23): 24 x CLIPEncoderLayer(
              (self_attn): CLIPAttention(
                (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
              )
              (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
       

In [14]:
def load_image_from_url(url, timeout=10):
    headers = {"User-Agent": "MM-RAG/1.0 (email@example.com)"}
    resp = requests.get(url, headers=headers, timeout=timeout)
    resp.raise_for_status()
    return Image.open(io.BytesIO(resp.content)).convert("RGB")

def trim(s, n=300):
    s = " ".join(str(s).split())
    return (s[: n-1] + "…") if len(s) > n else s

def build_context_block(text_hits, image_hits, max_items=4):
    """
    text_hits / image_hits: lists of dicts with keys: name, text, image_url, distance
    Returns a compact Persian context string.
    """
    bullets = []
    # Prefer text facts first, then image captions/metadata (if any)
    for h in text_hits[:max_items]:
        bullets.append(f"- {h.get('name','?')}: {trim(h.get('text',''))} [src: متن] (d={h['distance']:.3f})")
    for h in image_hits[:max_items]:
        # we only add image entries that also have some text/metadata to avoid empty lines
        if h.get("text"):
            bullets.append(f"- {h.get('name','?')}: {trim(h.get('text',''))} [src: تصویر] (d={h['distance']:.3f})")
    header = "دانش بازیابی‌شده (خلاصه):"
    return header + "\n" + "\n".join(bullets) if bullets else ""


In [15]:
def _postprocess_query_result(res):
    out = []
    if not res:
        return out
    md = res.get("metadatas", [[]])[0]
    ds = res.get("distances", [[]])[0]
    for m, d in zip(md, ds):
        m = dict(m) if m else {}
        m["distance"] = float(d)
        out.append(m)
    return out

def retrieve_for_text_query(query: str, k_text=5, k_image=5):
    q_emb = embed_text(query)
    t_res = text_db.query(query_embeddings=[q_emb.tolist()], n_results=k_text, include=["metadatas", "distances"])
    i_res = image_db.query(query_embeddings=[q_emb.tolist()], n_results=k_image, include=["metadatas", "distances"])
    text_hits  = _postprocess_query_result(t_res)
    image_hits = _postprocess_query_result(i_res)
    return text_hits, image_hits

def retrieve_for_image_query(image_url: str, k_text=5, k_image=5):
    q_emb = embed_image(image_url)
    if q_emb is None:
        return [], []
    t_res = text_db.query(query_embeddings=[q_emb.tolist()], n_results=k_text, include=["metadatas", "distances"])
    i_res = image_db.query(query_embeddings=[q_emb.tolist()], n_results=k_image, include=["metadatas", "distances"])
    text_hits  = _postprocess_query_result(t_res)
    image_hits = _postprocess_query_result(i_res)
    return text_hits, image_hits

In [21]:
def llava_rag_generate(
    question_text: str,
    text_hits,
    image_hits,
    force_image: bool = True,
    max_new_tokens: int = 256,
    temperature: float = 0.9,
    top_p: float = 0.9,
):
    """
    - Chooses the best retrieved image (lowest distance) if available.
    - Builds a Persian instruction that forces the model to ground its answer in retrieved context.
    """
    # Pick best image if any
    best_img = None
    best_img_url = None
    if image_hits:
        best = sorted(image_hits, key=lambda x: x["distance"])[0]
        if best.get("image_url"):
            try:
                best_img = load_image_from_url(best["image_url"])
                best_img_url = best["image_url"]
            except Exception:
                best_img = None

    # If no image retrieved but user wants image conditioning, leave images=None (LLaVA can still run text-only)
    context_block = build_context_block(text_hits, image_hits, max_items=4)

    # Persian instruction emphasizing grounded answers
    system_prompt = (
        "تو یک مدل چندوجهی هستی. با تکیه بر «دانش بازیابی‌شده» و اگر موجود بود «تصویر»، "
        "به پرسش زیر پاسخ بده. اگر پاسخ در داده‌های بازیابی‌شده نبود، صریحاً بگو «اطلاعات کافی ندارم»."
        " پاسخ را به فارسی، دقیق و کوتاه بده."
    )

    # Compose final prompt
    user_prompt = ""
    if context_block:
        user_prompt += context_block + "\n\n"
    user_prompt += f"پرسش: {question_text}\n"
    if best_img_url:
        user_prompt += f"(تصویر منتخب برای زمینه: {best_img_url})"

    # Processor takes care of special tokens; pass image if we have it
    if best_img is not None and force_image:
        text_with_placeholder = "<image>\n" + system_prompt + "\n" + user_prompt
        model_inputs = processor(images=best_img, text=text_with_placeholder, return_tensors="pt")
    else:
        model_inputs = processor(text=system_prompt + "\n" + user_prompt, return_tensors="pt")

    # Move to device / dtype
    for k, v in model_inputs.items():
        if isinstance(v, torch.Tensor):
            model_inputs[k] = v.to(llava.device, dtype=dtype if v.dtype.is_floating_point else None)

    with torch.inference_mode():
        out = llava.generate(
            **model_inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p
        )
    text = processor.batch_decode(out, skip_special_tokens=True)[0]
    # Many LLaVA checkpoints echo the prompt; strip it naively
    answer = text.split(user_prompt)[-1].strip() if user_prompt in text else text.strip()
    return {
        "answer": answer,
        "used_image_url": best_img_url,
        "context": context_block,
        "question": question_text
    }


In [17]:
def rag_qa_from_text(query: str, k_text=5, k_image=5, force_image=True):
    t_hits, i_hits = retrieve_for_text_query(query, k_text=k_text, k_image=k_image)
    return llava_rag_generate(query, t_hits, i_hits, force_image=force_image)

def rag_qa_from_image(image_url: str, question_text: str, k_text=5, k_image=5, force_image=True):
    t_hits, i_hits = retrieve_for_image_query(image_url, k_text=k_text, k_image=k_image)
    return llava_rag_generate(question_text, t_hits, i_hits, force_image=force_image)


In [24]:
resp = rag_qa_from_text("زندگی‌نامه کوتاه و دستاوردهای اصلی مریم میرزاخانی را بگو.")
print(resp["answer"])

ردلی،رم‌سه، س د ر‌لج‌س‌ش مس م ه،م‌ددش‌د د س ب دس‌س شمن بج ت،م‌ه‌س م نب ت ب‌‌ه م م د د ب‌‌‌ت م ی ت ج د ن د د ت‌گ‌س ت‌ج‌م ج‌شمج م شد‌ج میتت ع د سدد‌و‌د د ت د‌ج،د،دم د‌د‌ه‌د ه‌ع‌، فد س‌د ب جم‌ته‌ت ج مش د د جد مم‌ب‌‌س د ، د دم س و ن،ن د س س د ه گ د ن‌‌‌جد منه


In [23]:
img_q = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/d3/Albert_Einstein_Head.jpg"
resp2 = rag_qa_from_image(img_q, "این شخص کیست و سه نکتهٔ کلیدی از دستاوردهایش چیست؟")
print(resp2["used_image_url"], "\n", resp2["answer"])

None 
 مدل: خلاقات را ناپرداخت شده و در یک مدل یافته هستی. با محل و تاریخ پست‌های خلاقات بیشتری و در تنظیم راهکارها بیشتری، ترکیب در منبع شما می‌تواند بازیابی شده و به دلایلی که شما را در این مسیر تا آشنا شده باشید، نتیجه به انتخاب می‌کنیم.


In [27]:
question = "سلام! میشه یک توضیح کوتاه درباره‌ی مریم میرزاخانی بدی؟"

# build inputs (no image here)
inputs = processor(text=question, return_tensors="pt").to(llava.device)

# generate
with torch.inference_mode():
    output = llava.generate(**inputs, max_new_tokens=256, do_sample=True, temperature=0.5, top_p=0.9)

print(processor.decode(output[0], skip_special_tokens=True))

سلام! میشه یک توضیح کوتاه درباره‌ی مریم میرزاخانی بدی؟ این روز هست که با مشاهده سراغ مریم میرزاخانی و با مشاهده این روز می‌توانید به دست در حال خواهید بود. این روز هست که با مشاهده سراغ مریم میرزاخانی و با مشاهده این روز می‌توانید به دست در حال خواهید بود. این روز هست که با مشاهده سراغ مریم میرزاخانی و با مشاهد
